# 🏋️ NSSS Security Auditor - Colab Trainer

This notebook fine-tunes the Qwen2.5-Coder model using the NSSS Few-Shot Registry.

**Workflow:**
1.  **Prepare Data:** Downloads real CVEFixes dataset from HuggingFace and filters for Python.
2.  **Evaluate Baseline:** (Optional) Check base model performance before training.
3.  **Fine-tune:** Uses Unsloth (QLoRA) to train on T4/L4 GPU.
4.  **Evaluate Result:** Verify the new model's performance.
5.  **Save:** Exports the fine-tuned model to `outputs/qwen-security-model` on Drive.

**Prerequisites:**
-   **Runtime:** GPU (T4 is sufficient, A100 is faster).

In [ ]:
# @title 1. Configuration & Drive Mount
import os
from google.colab import drive

# @markdown ### 📂 Project Settings
DRIVE_ROOT = "/content/drive/MyDrive/NSSS_Project"  # @param {type:"string"}
GITHUB_REPO = "https://github.com/TCTri205/Neuro-Symbolic_Software_Security.git"  # @param {type:"string"}

# @markdown ### 🔄 Sync Options
UPDATE_FROM_GITHUB = True  # @param {type:"boolean"}

# Mount Drive
if not os.path.exists("/content/drive"):
    drive.mount("/content/drive")

print("✅ Google Drive mounted at /content/drive")

In [ ]:
# @title 2. Smart Sync (Drive <-> Colab)
import shutil
import subprocess


def run_cmd(cmd, cwd=None):
    print(f"⚡ Running: {cmd}")
    subprocess.run(cmd, shell=True, check=True, cwd=cwd)


# 1. Ensure Project exists and is a Git Repo
git_dir = os.path.join(DRIVE_ROOT, ".git")

if not os.path.exists(git_dir):
    print(f"📥 Initializing project at {DRIVE_ROOT}...")
    if not os.path.exists(DRIVE_ROOT):
        os.makedirs(DRIVE_ROOT, exist_ok=True)

    # Use git init + fetch to handle non-empty directories (Drive metadata)
    run_cmd("git init", cwd=DRIVE_ROOT)
    try:
        run_cmd(f"git remote add origin {GITHUB_REPO}", cwd=DRIVE_ROOT)
    except:
        run_cmd(f"git remote set-url origin {GITHUB_REPO}", cwd=DRIVE_ROOT)

    run_cmd("git fetch origin", cwd=DRIVE_ROOT)
    # Force checkout to overwrite any system files
    run_cmd("git reset --hard origin/main || git reset --hard origin/master", cwd=DRIVE_ROOT)
else:
    # 2. Optional Update
    if UPDATE_FROM_GITHUB:
        print("🔄 Updating code from GitHub...")
        run_cmd("git pull", cwd=DRIVE_ROOT)

# 3. Setup Workspace on Colab VM
WORKSPACE = "/content/app"

if os.path.exists(WORKSPACE):
    shutil.rmtree(WORKSPACE)

print("🚀 Copying code to Colab Runtime...")
shutil.copytree(
    DRIVE_ROOT,
    WORKSPACE,
    ignore=shutil.ignore_patterns("outputs", "data", "venv", ".git", "__pycache__"),
)

# 4. Link Data & Outputs to Drive
outputs_drive = os.path.join(DRIVE_ROOT, "outputs")
outputs_app = os.path.join(WORKSPACE, "outputs")

data_drive = os.path.join(DRIVE_ROOT, "data")
data_app = os.path.join(WORKSPACE, "data")

if not os.path.exists(outputs_drive):
    os.makedirs(outputs_drive)
if not os.path.exists(data_drive):
    os.makedirs(data_drive)

# Safe symlink creation
for drive_path, app_path in [(outputs_drive, outputs_app), (data_drive, data_app)]:
    if os.path.exists(app_path):
        if os.path.islink(app_path):
            os.unlink(app_path)
        else:
            shutil.rmtree(app_path)
    os.symlink(drive_path, app_path)

os.chdir(WORKSPACE)
print(f"📍 Working directory: {os.getcwd()}")

In [ ]:
# @title 3. Install Dependencies
print("📦 Installing Unsloth...")
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --progress-bar off
!pip install --no-deps "xformers<0.0.27" "trl<0.8.6" peft accelerate bitsandbytes --progress-bar off

if os.path.exists("requirements.txt"):
    !pip install -r requirements.txt --progress-bar off

print("✅ Environment Ready!")

In [ ]:
# @title 4. Prepare Data (Real CVEs)
# @markdown This step downloads CVEFixes from HuggingFace, filters for Python,
# @markdown and generates `data/few_shot_registry.json`.
# @markdown It may take a few minutes.

LIMIT = 2000  # @param {type:"integer"}

!python scripts/prepare_cve_data.py --limit $LIMIT

In [ ]:
# @title 4.1 Evaluate Baseline Model (Optional)
# @markdown Check how the base model performs BEFORE training.
# @markdown Only run this if you want to compare improvement.

RUN_BASELINE = True # @param {type:"boolean"}

if RUN_BASELINE:
    print("📊 Running Baseline Evaluation...")
    # Rename old report if exists
    !mv outputs/evaluation_report.json outputs/evaluation_report_old.json 2>/dev/null

    !python scripts/evaluate_model.py \
        --provider local \
        --model "unsloth/Qwen2.5-Coder-7B-Instruct-bnb-4bit" \
        --registry data/few_shot_registry.json

    # Save baseline report
    !mv outputs/evaluation_report.json outputs/report_baseline.json
    print("✅ Baseline saved to outputs/report_baseline.json")

In [ ]:
# @title 5. Run Training
# @markdown This step fine-tunes the model and saves it directly to `DRIVE_ROOT/outputs`.

!python scripts/train_model.py --registry data/few_shot_registry.json --output outputs/qwen-security-model

In [ ]:
# @title 6. Evaluate Fine-Tuned Model
# @markdown Compare the new model against the baseline.

print("📊 Running Final Evaluation...")
!python scripts/evaluate_model.py \
    --provider local \
    --model "outputs/qwen-security-model" \
    --registry data/few_shot_registry.json

print("\n✅ Evaluation Complete!")